<a href="https://colab.research.google.com/github/hwankook/coding-test/blob/main/algorithms/%EA%B0%80%EC%9E%A5_%ED%81%B0_%EC%82%BC%EA%B0%81%ED%98%95_%EB%8D%A9%EC%96%B4%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
from collections import deque

def solution(grid):
    if not grid or not grid[0]:
        return 0
    n, m = len(grid), len(grid[0])

    # 삼각형을 노드로 간주하여 인접 리스트 생성
    adj = [[] for _ in range(n * m * 2)]

    # 다음 삼각형을 찾는 함수
    def get_next_triangle(i, j, direction):
        if direction == 'right':
            if j + 1 >= m:
                return None
            return (i, j + 1, 0)
        elif direction == 'left':
            if j - 1 < 0:
                return None
            return (i, j - 1, 1)
        elif direction == 'bottom':
            if i + 1 >= n:
                return None
            cell_below = grid[i + 1][j]
            if cell_below == -1:
                return (i + 1, j, 1)  # \ 대각선, 위쪽은 삼각형 1
            else:
                return (i + 1, j, 0)  # / 대각선, 위쪽은 삼각형 0
        elif direction == 'top':
            if i - 1 < 0:
                return None
            cell_above = grid[i - 1][j]
            if cell_above == -1:
                return (i - 1, j, 0)  # \ 대각선, 아래쪽은 삼각형 0
            else:
                return (i - 1, j, 1)  # / 대각선, 아래쪽은 삼각형 1
        return None

    # 인접 리스트 구축
    for i in range(n):
        for j in range(m):
            for k in range(2):
                directions = []
                if grid[i][j] == -1:
                    if k == 0:
                        directions = ['left', 'bottom']  # \ 대각선, 왼쪽-아래
                    else:
                        directions = ['top', 'right']    # \ 대각선, 위-오른쪽
                else:
                    if k == 0:
                        directions = ['top', 'left']     # / 대각선, 위-왼쪽
                    else:
                        directions = ['bottom', 'right'] # / 대각선, 아래-오른쪽
                idx = 2 * (i * m + j) + k
                for d in directions:
                    nt = get_next_triangle(i, j, d)
                    if nt is not None:
                        ni, nj, nk = nt
                        adj[idx].append(2 * (ni * m + nj) + nk)

    # 셀 단위 방문 체크
    visited = [False] * (n * m)
    max_size = 0

    # BFS로 최대 연결된 셀 크기 탐색
    for i in range(n):
        for j in range(m):
            for k in range(2):
                start_idx = i * m + j
                if visited[start_idx]:
                    continue
                queue = deque([(i, j, k)])
                visited_cells = set([(i, j)])
                current_size = 1

                while queue:
                    ci, cj, ck = queue.popleft()
                    current_linear = 2 * (ci * m + cj) + ck
                    for next_idx in adj[current_linear]:
                        ni, nj = next_idx // 2 // m, (next_idx // 2) % m
                        if (ni, nj) not in visited_cells:
                            visited_cells.add((ni, nj))
                            queue.append((ni, nj, next_idx % 2))
                            current_size += 1

                max_size = max(max_size, current_size)
                for ci, cj in visited_cells:
                    visited[ci * m + cj] = True

    return max_size

# 테스트 케이스 실행
if __name__ == "__main__":
    test_cases = [
        ([[-1, -1, -1], [1, 1, -1], [1, 1, 1]], 5),
        ([[1, -1, 1], [-1, 1, -1]], 4),
        ([[1]], 1),
        ([[-1]], 1),
        ([[1, 1, 1]], 2),
        ([[-1, -1, -1]], 2),
        ([[1], [-1], [1], [1]], 2),
        ([[1, 1], [1, 1]], 3),
        ([[-1, -1], [-1, -1]], 3),
        ([[1, -1], [-1, 1]], 4),
        ([[1, -1, 1, -1], [-1, 1, -1, 1], [1, -1, 1, -1]], 4),
        ([[-1, -1, -1], [1, 1, -1], [1, 1, 1]], 5),
        ([[1, 1, 1], [1, 1, 1], [1, 1, 1]], 5),
        ([[-1, -1, -1], [-1, -1, -1], [-1, -1, -1]], 5),
        ([[1, -1, 1, -1, 1]], 2),
        ([[-1, -1, -1], [1, 1, -1], [-1, 1, -1]], 6),
        ([[1, -1, 1, -1], [1, 1, -1, -1], [-1, -1, 1, 1], [-1, -1, 1, -1]], 14),
    ]

    for i, (grid, expected) in enumerate(test_cases):
        result = solution(grid)
        test_result = True if result == expected else False
        if not test_result:
            print(f"Test {i + 1}: {test_result} (Result: {result}, Expected: {expected})")